# From Detecting Changes in Sentinel-1 Imagery (Part 1)

In [53]:
import ee
 
# Trigger the authentication flow.
ee.Authenticate()
 
# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AfgeXvv4IJh2wnntnAhlVirzksYIXfNuJi_FjGhR9YzsyyTVeWgYUewPrSs

Successfully saved authorization token.


In [54]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
%matplotlib inline

In [ ]:
# Make interactive maps using folium.
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name = name,
        overlay = True,
        control = True
    ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [56]:
# Define new region

geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              8.534317016601562,
              50.021637833966786
            ],
            [
              8.530540466308594,
              49.99780882512238
            ],
            [
              8.564186096191406,
              50.00663576154257
            ],
            [
              8.578605651855469,
              50.019431940583104
            ],
            [
              8.534317016601562,
              50.021637833966786
            ]
          ]
        ]
      }
    }
  ]
}
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi_sub = ee.Geometry.Polygon(coords)

In [57]:
# Subset an image on a region by using the property 
# .clip(region), where region being a Geometry object

ffa_db = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD') # dB
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2020-08-01'), ee.Date('2020-08-31')) 
                       .first() 
                       .clip(aoi_sub))
ffa_fl = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT') # linear scale
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2020-08-01'), ee.Date('2020-08-31')) 
                       .first() 
                       .clip(aoi_sub))

In [58]:
# List  band names, fetching the result from the GEE
# servers with the getInfo() class method:

ffa_db.bandNames().getInfo()

['VV', 'VH', 'angle']

In [59]:
# Show VV band with standard physical upper and lower
# limits of detection

url = ffa_db.select('VV').getThumbURL({'min': -20, 'max': 0})
disp.Image(url=url, width=800)

## Extract pixel values and examine distributions

In [60]:
# Using standard reducers from the GEE library we can
# easily calculate a histogram and estimate the first
# two moments (mean and variance) of the pixels in the
# polygon aoi_sub , again retrieving the results from
# the servers with getInfo().

d = dict.fromkeys(['VV', 'VH', 'VV_dev', 'VH_dev', 'angle'])

for i,pol in enumerate(['VV', 'VH']):
    # hist = ffa_db.select(f'{pol}').reduceRegion(
    #    ee.Reducer.fixedHistogram(0, 0.5, 500),aoi_sub).get(f'{pol}').getInfo()
    d[pol] = ffa_db.select(f'{pol}').reduceRegion(
        ee.Reducer.mean(), aoi_sub).get(f'{pol}').getInfo()
    d[f'{pol}_dev'] = np.sqrt(ffa_db.select(f'{pol}').reduceRegion(
        ee.Reducer.variance(), aoi_sub).get(f'{pol}').getInfo())
    
d['angle'] = ffa_db.select('angle').reduceRegion(
    ee.Reducer.mean(), aoi_sub).get('angle').getInfo()

In [61]:
d

{'VV': -10.051188447166275,
 'VH': -15.935466061188578,
 'VV_dev': 2.1164268646315056,
 'VH_dev': 2.220589022985799,
 'angle': 41.944610595703125}

At this level, the timestamp and passing (asc/desc) are still missing

In [62]:
import time

In [63]:
info = ffa_db.getInfo(); info

# Relevant infos:
#
# platform_number
# instrumentMode
# relativeOrbitNumber_stop
# orbitProperties_pass

{'type': 'Image',
 'bands': [{'id': 'VV',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [347, 266],
   'origin': [7157, 16014],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 394777.3455817547, 0, -10, 5701290.229369663]},
  {'id': 'VH',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [347, 266],
   'origin': [7157, 16014],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 394777.3455817547, 0, -10, 5701290.229369663]},
  {'id': 'angle',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1, 1],
   'origin': [14, 7],
   'crs': 'EPSG:32632',
   'crs_transform': [-12997.77519209648,
    -3350.910727790906,
    685033.6771857691,
    2138.4057794939727,
    -20077.23388737999,
    5655639.081878104]}],
 'id': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20200803T053414_20200803T053439_033738_03E909_0119',
 'version': 1668410218758401,
 'properties': {'GRD_Post_Processing_start': 1596441587213,
  'slic

In [64]:
info['properties']['platform_number']

'A'

In [65]:
time_start = info['properties']['system:time_start']
time.strftime('%x', time.gmtime(time_start/1000))

'08/03/20'

And this is how you do it!

# From Part 2

In [30]:
# Filter images by metadata and produce an images' JS collection

im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                .filterBounds(aoi)
                .filterDate(ee.Date('2020-08-01'),ee.Date('2020-08-31'))
                .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
                .filter(ee.Filter.eq('relativeOrbitNumber_start', 15))
                .sort('system:time_start'))

# Retrieve the acquisition times in the collection

import time
acq_times = im_coll.aggregate_array('system:time_start').getInfo()
[time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]

['08/05/20', '08/11/20', '08/17/20', '08/23/20', '08/29/20']

In [31]:
# Example: choose the first and the second images and compute the ratio on VV bands

im_list = im_coll.toList(im_coll.size())
im1 = ee.Image(im_list.get(0)).select('VV').clip(aoi_sub)
im2 = ee.Image(im_list.get(1)).select('VV').clip(aoi_sub)
ratio = im1.divide(im2)
url = ratio.getThumbURL({'min': 0, 'max': 10})
disp.Image(url=url, width=800)

# From description of product: Sentinel-1 SAR GRD: C-band Synthetic Aperture Radar Ground Range Detected, log scaling

Ref. https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD

## JavaScript code

# From End-to-end GEE

Ref. https://courses.spatialthoughts.com/end-to-end-gee.html

## Javascript

### EE API objects basics

### Mapping w/ .map(): applying single function to the whole Image Collection

### Computations w/ .reduce(): compute statistics on large amount of inputs

When writing parallel computing code, a Reduce operation allows you to compute statistics on a large amount of inputs. In Earth Engine, you need to run reduction operation when creating composites, calculating statistics, doing regression analysis etc. The Earth Engine API comes with a large number of built-in reducer functions (such as ee.Reducer.sum(), ee.Reducer.histogram(), ee.Reducer.linearFit() etc.) that can perform a variety of statistical operations on input data. You can run reducers using the reduce() function. Earth Engine supports running reducers on all data structures that can hold multiple values, such as Images (reducers run on different bands), ImageCollection, FeatureCollection, List, Dictionary etc. The script below introduces basic concepts related to reducers.

### Time-series creation

# Other snippets